In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateScheduler,
)
import numpy as np
import cv2
import random
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16
from sklearn.utils.class_weight import compute_class_weight

2024-04-14 10:29:16.475592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 10:29:16.653715: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 10:29:17.457776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-14 10:29:17.457897: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
import os

# Set environment variable for TensorFlow
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import tensorflow as tf

# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Set memory growth before initializing GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Handle potential errors here
        print(e)

1 Physical GPUs, 1 Logical GPUs


2024-04-14 10:29:21.748984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 10:29:22.183295: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0


In [3]:
import numpy as np
import random
import tensorflow as tf
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

## with augmentation

In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateScheduler,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import SGD
import numpy as np
import cv2
import random

# Load data from file
X_train_data = np.load(
    "/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/X_train.npy"
)
X_test_data = np.load("/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/X_test.npy")
y_train_data = np.load(
    "/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/y_train.npy"
)
y_test_data = np.load("/workspaces/Diabetic-Retinopathy/aws_s3/messidor_512/y_test.npy")

# Augmentation functions


def rotate_image(image, angle):
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def flip_image(image):
    flip_variations = [
        (False, False),  # No flip
        (True, False),  # Horizontal flip
        (False, True),  # Vertical flip
        (True, True),  # Both flips
    ]

    random.shuffle(flip_variations)
    augmented_images = []

    for flip_horizontal, flip_vertical in flip_variations[:4]:
        if flip_horizontal and flip_vertical:
            augmented_images.append(cv2.flip(image, -1))  # horizontal and vertical
        elif flip_horizontal:
            augmented_images.append(cv2.flip(image, 1))  # horizontal
        elif flip_vertical:
            augmented_images.append(cv2.flip(image, 0))  # vertical
        else:
            augmented_images.append(image)  # No flip

    return augmented_images


def shear_image(image, shear_range):
    height, width = image.shape[:2]
    shear_value = random.uniform(-shear_range, shear_range)

    if shear_value < 0:
        shear_matrix = np.array([[1, -shear_value, 0], [0, 1, 0]])
    else:
        shear_matrix = np.array([[1, shear_value, 0], [0, 1, 0]])

    sheared_image = cv2.warpAffine(image, shear_matrix, (width, height))
    return sheared_image


def translate_image(image, translate_range):
    height, width = image.shape[:2]

    max_shift_x = int(width * 0.1)
    max_shift_y = int(height * 0.1)

    translate_x = random.randint(-max_shift_x, max_shift_x)
    translate_y = random.randint(-max_shift_y, max_shift_y)

    translation_matrix = np.array(
        [[1, 0, translate_x], [0, 1, translate_y]], dtype=np.float32
    )

    translated_image = cv2.warpAffine(image, translation_matrix, (width, height))

    return translated_image


def adjust_brightness(image, brightness_range):
    brightness_factor = 1.0 + random.uniform(-brightness_range, brightness_range)
    adjusted_image = np.clip(image * brightness_factor, 0.25, 255).astype(np.uint8)
    return adjusted_image


def augmented_fn2(image, flag):
    augmented_images = []

    if flag == 1:
        for _ in range(4):
            angle = random.uniform(-35, 35)
            image_r = rotate_image(image, angle)
            augmented_images.append(image_r)

        image_f = flip_image(image)
        augmented_images.extend(image_f)

        for _ in range(4):
            image_s = shear_image(image, shear_range=0.15)
            augmented_images.append(image_s)

        for _ in range(4):
            image_t = translate_image(image, translate_range=0.1)
            augmented_images.append(image_t)

        for _ in range(4):
            image_b = adjust_brightness(image, brightness_range=0.25)
            augmented_images.append(image_b)
    else:
        for _ in range(4):
            angle = random.uniform(-35, 35)
            image_r = rotate_image(image, angle)
            augmented_images.append(image_r)

        image_f = flip_image(image)
        augmented_images.extend(image_f)

        for _ in range(4):
            image_s = shear_image(image, shear_range=0.15)
            augmented_images.append(image_s)

        for _ in range(4):
            image_t = translate_image(image, translate_range=0.1)
            augmented_images.append(image_t)

    return np.array(augmented_images)


# Apply augmentation to training data
X_train_augmented = []
y_train_augmented = []

for image, label in zip(X_train_data, y_train_data):
    augmented_images = augmented_fn2(
        image, flag=1
    )  # Set flag to 1 for including brightness augmentation
    X_train_augmented.extend(augmented_images)
    y_train_augmented.extend([label] * len(augmented_images))

# Convert lists to arrays
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

# Load pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(512, 512, 3)
)

# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the pre-trained model
model = Sequential(
    [
        base_model,
        Flatten(),
        Dense(512, activation="relu"),
        Dropout(0.5),
        Dense(256, activation="relu"),
        Dropout(0.5),
        Dense(4, activation="softmax"),  # Output layer with 4 classes
    ]
)

# Define optimizer
optimizer = SGD(learning_rate=1e-1, momentum=0.9)

# Compile the model
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Define custom learning rate scheduler


def triangular_lr(epoch, lr):
    base_lr = 1e-4
    max_lr = 1e-1
    step_size = 8
    cycle = tf.floor(1 + epoch / (2 * step_size))
    x = tf.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * tf.maximum(0, (1 - x))
    return lr


# Define callbacks
early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_accuracy", save_best_only=True
)

lr_scheduler = LearningRateScheduler(triangular_lr)

# Train the model
history = model.fit(
    X_train_augmented,
    y_train_augmented,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint, lr_scheduler],
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_data, y_test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

2024-04-14 10:30:39.982781: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14633926656 exceeds 10% of free system memory.
2024-04-14 10:30:55.917387: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 13.63GiB (rounded to 14633926656)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-04-14 10:30:55.917446: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2024-04-14 10:30:55.917467: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 105, Chunks in use: 105. 26.2KiB allocated for chunks. 26.2KiB in use in bin. 7.7KiB client-requested in use in bin.
2024-04-14 10:30:55.917482: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chu

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.